In [0]:
import numpy as np
import os
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from datetime import timezone
import datetime
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import requests
import pprint

In [0]:
comp = "AAPL"
apiKey = "SU8EEDGKD0M354K0"
url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&apikey={}&outputsize=full".format(comp, apiKey)

In [0]:
r = requests.get(url)
data = r.json()
pprint.pprint(data)
#confirmed that data request generates a dictionary of kinf:
"""
{'Meta Data': {'1. Information': 'Weekly Prices (open, high, low, close) and '
                                 'Volumes',
               '2. Symbol': 'TSLA',
               '3. Last Refreshed': '2020-02-21',
               '4. Time Zone': 'US/Eastern'},
 'Weekly Time Series': {'2010-07-09': {'1. open': '20.0000',
                                       '2. high': '20.0000',
                                       '3. low': '14.9800',
                                       '4. close': '17.4000',
                                       '5. volume': '25550600'},
"""


Streaming output truncated to the last 5000 lines.
                                        '2. high': '162.0000',
                                        '3. low': '159.9300',
                                        '4. close': '161.4700',
                                        '5. adjusted close': '155.7834',
                                        '6. volume': '25279674',
                                        '7. dividend amount': '0.0000',
                                        '8. split coefficient': '1.0000'},
                         '2017-08-29': {'1. open': '160.1000',
                                        '2. high': '163.1200',
                                        '3. low': '160.0000',
                                        '4. close': '162.9100',
                                        '5. adjusted close': '157.1727',
                                        '6. volume': '29307862',
                                        '7. dividend amount': '0.0000',
             

"\n{'Meta Data': {'1. Information': 'Weekly Prices (open, high, low, close) and '\n                                 'Volumes',\n               '2. Symbol': 'TSLA',\n               '3. Last Refreshed': '2020-02-21',\n               '4. Time Zone': 'US/Eastern'},\n 'Weekly Time Series': {'2010-07-09': {'1. open': '20.0000',\n                                       '2. high': '20.0000',\n                                       '3. low': '14.9800',\n                                       '4. close': '17.4000',\n                                       '5. volume': '25550600'},\n"

In [0]:
stock_data = pd.DataFrame.from_dict(data["Time Series (Daily)"])
transposed_stock_data = stock_data.T
transposed_stock_data.index = pd.to_datetime(transposed_stock_data.index)
transposed_stock_data['year'] = transposed_stock_data.index.year
transposed_stock_data['month'] = transposed_stock_data.index.month
transposed_stock_data['day'] = transposed_stock_data.index.day
transposed_stock_data.reset_index(inplace=True)
transposed_stock_data.drop(transposed_stock_data.index)
transposed_stock_data.drop(["3. low"], axis = 1, inplace = True)
transposed_stock_data.drop(["5. adjusted close"], axis = 1, inplace = True)
transposed_stock_data.drop(["6. volume", "7. dividend amount", "8. split coefficient"], axis = 1, inplace = True)
transposed_stock_data.drop(["2. high"], axis = 1, inplace = True)
for j in [(int(datetime.date(transposed_stock_data["year"][i], transposed_stock_data["month"][i], transposed_stock_data["day"][i]).strftime('%s')) for i in range(transposed_stock_data.shape[0]))]:
  transposed_stock_data["utc"] = list(j)
# transposed_stock_data["utc"] = ((int(datetime.date(transposed_stock_data["year"][i], transposed_stock_data["month"][i], transposed_stock_data["day"][i]).strftime('%s')) for i in range(transposed_stock_data.shape[0])))
transposed_stock_data

,index,1. open,4. close,year,month,day,utc
0,2020-02-21,318.6200,313.0500,2020,2,21,1582243200
1,2020-02-20,322.6300,320.3000,2020,2,20,1582156800
2,2020-02-19,320.0000,323.6200,2020,2,19,1582070400
3,2020-02-18,315.3600,319.0000,2020,2,18,1581984000
4,2020-02-14,324.7300,324.9500,2020,2,14,1581638400
...,...,...,...,...,...,...,...
5027,2000-02-28,110.1200,113.2500,2000,2,28,951696000
5028,2000-02-25,114.8100,110.3700,2000,2,25,951436800
5029,2000-02-24,117.3100,115.2000,2000,2,24,951350400
5030,2000-02-23,113.2300,116.2500,2000,2,23,951264000


In [0]:
for j in [(int(datetime.date(transposed_stock_data["year"][i], transposed_stock_data["month"][i], transposed_stock_data["day"][i]).strftime('%s')) for i in range(transposed_stock_data.shape[0]))]:
  print(list(j))

[1582243200, 1582156800, 1582070400, 1581984000, 1581638400, 1581552000, 1581465600, 1581379200, 1581292800, 1581033600, 1580947200, 1580860800, 1580774400, 1580688000, 1580428800, 1580342400, 1580256000, 1580169600, 1580083200, 1579824000, 1579737600, 1579651200, 1579564800, 1579219200, 1579132800, 1579046400, 1578960000, 1578873600, 1578614400, 1578528000, 1578441600, 1578355200, 1578268800, 1578009600, 1577923200, 1577750400, 1577664000, 1577404800, 1577318400, 1577145600, 1577059200, 1576800000, 1576713600, 1576627200, 1576540800, 1576454400, 1576195200, 1576108800, 1576022400, 1575936000, 1575849600, 1575590400, 1575504000, 1575417600, 1575331200, 1575244800, 1574985600, 1574812800, 1574726400, 1574640000, 1574380800, 1574294400, 1574208000, 1574121600, 1574035200, 1573776000, 1573689600, 1573603200, 1573516800, 1573430400, 1573171200, 1573084800, 1572998400, 1572912000, 1572825600, 1572566400, 1572480000, 1572393600, 1572307200, 1572220800, 1571961600, 1571875200, 1571788800, 157

In [0]:
os.listdir()

['.config', 'stock_market.csv', 'sample_data']

In [0]:
transposed_stock_data.to_csv("stock_market.csv", index = None, header=True)